<a href="https://colab.research.google.com/github/chitransh-singhal/Deep-Learning/blob/master/VGG16_Digit_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## MNist Classification

MNIST Digit Classification using VGG16-Transfer Learning

Import Essential Modules

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from keras.utils import to_categorical, plot_model
from keras.preprocessing import image
from keras import models, layers

Loading the MNIST Data

In [3]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

11490434/11490434 [==============================] - 0s 0us/step


In [4]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)
print(y_train)

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)
[5 0 4 ... 5 6 8]


Processing the data to make it compatible with VGG16 model

In [5]:
x_train = np.dstack([x_train]*3)
x_test = np.dstack([x_test]*3)
x_train.shape, x_test.shape

((60000, 28, 84), (10000, 28, 84))

Reshape images as per the tensor format required by tensorfow

In [6]:
x_train = x_train.reshape(-1,28,28,3)
x_test = x_test.reshape(-1,28,28,3)
x_train.shape, x_test.shape

((60000, 28, 28, 3), (10000, 28, 28, 3))

Resizing the images into 48*48 as required for VGG16

In [7]:
from keras.preprocessing.image import img_to_array, array_to_img

x_train = np.asarray([img_to_array(array_to_img(im, scale=False).
                                   resize((48,48))) for im in x_train])
x_test = np.asarray([img_to_array(array_to_img(im, scale=False).
                                   resize((48,48))) for im in x_test])
x_train.shape, x_test.shape

((60000, 48, 48, 3), (10000, 48, 48, 3))

Preparing the data for VGG16 model

In [8]:
from sklearn.model_selection import train_test_split

x=[]
x=x_train
y=[]
y=y_train

y=to_categorical(y)
x=np.array(x)

x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=5)

Defing the model using Transfer Learning

In [9]:
from keras.applications import VGG16
model_vgg = VGG16(weights='imagenet')

553467096/553467096 [==============================] - 8s 0us/step


Adding the input layer to the model

In [10]:
inp_layer = layers.Input(shape=(48,48,3))
model_vgg = VGG16(weights='imagenet', input_tensor=inp_layer, include_top=False)

58889256/58889256 [==============================] - 1s 0us/step


Adding the output layers

In [11]:
last_layer = model_vgg.output
flatten = layers.Flatten()(last_layer)
dense1 = layers.Dense(100, activation='relu')(flatten)
dense1 = layers.Dense(100, activation='relu')(flatten)
dense1 = layers.Dense(100, activation='relu')(flatten)
output_layer = layers.Dense(10, activation='softmax')(flatten)

model = models.Model(inputs=inp_layer, outputs = output_layer)

We are making all the layers intrainable except the last layer

In [12]:
for layer in model.layers[:-1]:
  layer.trainable = False

Compile the model

In [13]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
print("Model compilation completed.")
model.summary()

Model compilation completed.
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 48, 48, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 48, 48, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 48, 48, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 24, 24, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 24, 24, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 24, 24, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (No

Model fitting

In [14]:
history = model.fit(x_train, y_train, epochs=20, batch_size=128, verbose=True, validation_data=(x_test, y_test))

Epoch 1/20
375/375 [==============================] - 1461s 4s/step - loss: 2.2129 - accuracy: 0.7118 - val_loss: 0.6203 - val_accuracy: 0.8560
Epoch 2/20
375/375 [==============================] - 1464s 4s/step - loss: 0.4897 - accuracy: 0.8774 - val_loss: 0.4010 - val_accuracy: 0.8965
Epoch 3/20
375/375 [==============================] - 1452s 4s/step - loss: 0.3534 - accuracy: 0.9023 - val_loss: 0.3334 - val_accuracy: 0.9043
Epoch 4/20
375/375 [==============================] - 1451s 4s/step - loss: 0.2970 - accuracy: 0.9143 - val_loss: 0.3055 - val_accuracy: 0.9104
Epoch 5/20
375/375 [==============================] - 1453s 4s/step - loss: 0.2685 - accuracy: 0.9205 - val_loss: 0.2808 - val_accuracy: 0.9182
Epoch 6/20
375/375 [==============================] - 1449s 4s/step - loss: 0.2549 - accuracy: 0.9230 - val_loss: 0.2679 - val_accuracy: 0.9218
Epoch 7/20
375/375 [==============================] - 1454s 4s/step - loss: 0.2454 - accuracy: 0.9246 - val_loss: 0.2652 - val_accuracy:

Evaluating the model

In [15]:
loss, accuracy = model.evaluate(x_test, y_test, verbose=1)

375/375 [==============================] - 287s 765ms/step - loss: 0.2640 - accuracy: 0.9230


In [16]:
print("The loss is ",loss,"And the accuracy is ", accuracy)

The loss is  0.26401281356811523 And the accuracy is  0.9229999780654907
